In [315]:
import pandas as pd
import os
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, pipeline
import math
import torch
import numpy as np
from itertools import product

input_dir = '../data/inputs'
output_dir = '../data/outputs/s0_1007'
results_dir = '../data/results_debias'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [322]:
job_df = pd.read_csv(os.path.join(input_dir, 'female_ratios.csv'))
jobs = job_df['job'].to_list()

for f in os.listdir(output_dir):
    df = pd.read_csv(os.path.join(output_dir, f))
    if 'job' not in df.columns:
        df['job'] = jobs
        df.to_csv(os.path.join(output_dir, f), index=False)

In [332]:
for f in os.listdir(output_dir):
    df = pd.read_csv(os.path.join(output_dir, f))
    for col in df.columns:
        if 'met' in col:
            print(f)
            break

mistral-7b_low-2.csv
llama2-7b_medium-4.csv
mistral-7b_high-5.csv
llama3-8b-instruct_low-1.csv
mistral-7b_high-6.csv
llama3-8b-instruct_low-2.csv
llama3-8b_high-6.csv
llama2-7b_medium-3.csv
llama3-8b_high-5.csv
llama3-8b-instruct_high-6.csv
llama3-8b_low-1.csv
mistral-7b-instruct_low-2.csv
mistral-7b-instruct_medium-3.csv
llama3-8b-instruct_high-5.csv
llama3-8b_medium-3.csv
llama3-8b_low-2.csv
mistral-7b-instruct_high-5.csv
llama2-7b-instruct_low-2.csv
mistral-7b-instruct_high-6.csv
llama3-8b_medium-4.csv
llama2-7b-instruct_low-1.csv
mistral-7b-instruct_medium-4.csv
llama2-7b_high-5.csv
mistral-7b_medium-3.csv
llama2-7b-instruct_medium-3.csv
llama2-7b_high-6.csv
llama3-8b-instruct_medium-4.csv
llama2-7b_low-2.csv
llama2-7b-instruct_medium-4.csv
mistral-7b_medium-4.csv
llama2-7b-instruct_high-6.csv
llama2-7b-instruct_high-5.csv
llama3-8b-instruct_medium-3.csv


In [316]:
model_str_map = {
    'llama3-8b': 'Llama-3-8B',
    'llama3-8b-instruct': 'Llama-3-8B-Instruct',
    'mistral-7b': 'Mistral-7B',
    'mistral-7b-instruct': 'Mistral-7B-Instruct',
    'llama2-7b': 'Llama-2-7B',
    'llama2-7b-instruct': 'Llama-2-7B-Instruct',
    'gemma-7b': 'Gemma-7B',
    'gemma-7b-instruct': 'Gemma-7B-Instruct',
    'gemma-2-9b': 'Gemma-2-9B',
    'gemma-2-9b-instruct': 'Gemma-2-9B-Instruct',
}
model_strs = ['llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-instruct', 'gemma-7b', 'gemma-7b-instruct', 'gemma-2-9b', 'gemma-2-9b-instruct']
model_strs = sorted(model_strs, key=len, reverse=True)

In [317]:
female_ratios = pd.read_csv(os.path.join(input_dir, 'female_ratios.csv'))
# for model in  llama3-8b llama3-8b-instruct mistral-7b mistral-7b-instruct llama2-7b llama2-7b-chat


prompt_ids = ['none','low-1','low-2','medium-3','medium-4','high-5','high-6']
prompt_id_mapping = {pid: idx for idx, pid in enumerate(prompt_ids)}

df_list = []
for f in os.listdir(output_dir):
    if "conv" not in f and "gender" not in f and 'gpt2' not in f:
    # if 'conversation.csv' in f:
    
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])
        for col in df.columns:
            if '.' in col:
                df.drop(columns=[col.replace('.1','')],inplace=True)
                df.rename(columns={col:col.replace('.1','')},inplace=True)

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['debiasing_prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f
        df.to_csv(os.path.join(output_dir, f), index=False)

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)


        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        # if 'male_met-met' in df.columns:
        #     col_ends = ['met-met', 'friend', 'talk-met']
        #     # Compute averages for male, female, and diverse columns
        #     male_cols = ['male_' + end for end in col_ends]
        #     female_cols = ['female_' + end for end in col_ends]
        #     diverse_cols = ['diverse_' + end for end in col_ends]
        # 
        # 
        #     grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
        #     grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
        #     grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
        #     grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)
        # 
        #     
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'implicit.csv'), index=False)

KeyError: 'job'

In [318]:
df

,llama3-8b-instruct_male_implicit0_prob,llama3-8b-instruct_female_implicit0_prob,llama3-8b-instruct_diverse_implicit0_prob,llama3-8b-instruct_male_implicit0,llama3-8b-instruct_female_implicit0,llama3-8b-instruct_diverse_implicit0,llama3-8b-instruct_male_implicit1_prob,llama3-8b-instruct_female_implicit1_prob,llama3-8b-instruct_diverse_implicit1_prob,llama3-8b-instruct_male_implicit1,...,llama3-8b-instruct_female_implicit4_prob,llama3-8b-instruct_diverse_implicit4_prob,llama3-8b-instruct_male_implicit4,llama3-8b-instruct_female_implicit4,llama3-8b-instruct_diverse_implicit4,female_ratio,female_dominated,debiasing_prompt_id,model,conversation
0,0.000078,0.000398,0.000146,0.1258,0.6391,0.2351,0.000025,0.000397,0.000146,0.0446,...,4.102295e-03,9.743805e-04,0.0587,0.7607,0.1807,98.2,True,0,llama3-8b-instruct,False
1,0.000007,0.000265,0.000001,0.0259,0.9703,0.0038,0.000004,0.000208,0.000017,0.0166,...,2.443443e-03,1.752561e-05,0.0309,0.9622,0.0069,96.8,True,0,llama3-8b-instruct,False
2,0.000037,0.000425,0.000048,0.0728,0.8336,0.0935,0.000009,0.000293,0.000054,0.0264,...,4.183170e-03,3.225716e-04,0.0594,0.8732,0.0673,94.6,True,0,llama3-8b-instruct,False
3,0.000033,0.000263,0.000030,0.1023,0.8046,0.0931,0.000010,0.000214,0.000115,0.0305,...,1.815258e-03,2.285074e-05,0.0369,0.9511,0.0120,92.5,True,0,llama3-8b-instruct,False
4,0.000072,0.000368,0.000087,0.1373,0.6971,0.1656,0.000020,0.000289,0.000121,0.0458,...,2.656109e-03,5.230189e-04,0.0907,0.7597,0.1496,92.4,True,0,llama3-8b-instruct,False
5,0.000056,0.000465,0.000049,0.0975,0.8164,0.0861,0.000020,0.000397,0.000100,0.0382,...,4.598859e-03,2.289636e-04,0.0453,0.9094,0.0453,92.0,True,0,llama3-8b-instruct,False
6,0.000070,0.000517,0.000027,0.1139,0.8415,0.0446,0.000011,0.000242,0.000089,0.0311,...,4.599534e-03,6.163377e-05,0.0497,0.9377,0.0126,91.3,True,0,llama3-8b-instruct,False
7,0.000074,0.000585,0.000027,0.1083,0.8519,0.0398,0.000029,0.000425,0.000084,0.0538,...,4.580019e-03,4.001976e-04,0.1236,0.8060,0.0704,90.4,True,0,llama3-8b-instruct,False
8,0.000045,0.000398,0.000037,0.0931,0.8297,0.0772,0.000011,0.000225,0.000120,0.0314,...,2.816897e-03,7.506772e-05,0.0463,0.9290,0.0248,90.0,True,0,llama3-8b-instruct,False
9,0.000383,0.000763,0.000193,0.2864,0.5696,0.1440,0.000162,0.000566,0.000365,0.1483,...,3.928212e-03,1.125452e-03,0.3625,0.4955,0.1420,89.8,True,0,llama3-8b-instruct,False


In [305]:
df_list = []
for f in os.listdir(output_dir):
    if 'genderquestion.csv' in f and 'gpt2' not in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])
        for col in df.columns:
            if '.' in col:
                df.drop(columns=[col.replace('.1','')],inplace=True)
                df.rename(columns={col:col.replace('.1','')},inplace=True)


        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['debiasing_prompt_id'] = prompt_id
        
        # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f
        df.to_csv(os.path.join(output_dir, f), index=False)

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()

        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'explicit.csv'), index=False)

In [306]:
df_implicit = pd.read_csv(os.path.join(results_dir, 'implicit.csv'))
df_explicit = pd.read_csv(os.path.join(results_dir, 'explicit.csv'))
# if any column starts with a space, remove it
df_implicit.columns = df_implicit.columns.str.strip()
df_explicit.columns = df_explicit.columns.str.strip()

for model, group_df in df_implicit.groupby('model'):
    # Save the DataFrame to a CSV file
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'implicit.csv'), index=False)

for model, group_df in df_explicit.groupby('model'):
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'explicit.csv'), index=False)


In [307]:
prompt_id_map = {
    0: 'None',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6'
}

abstraction_levels = {
    0: '',
    1: 'High',
    2: 'High',
    3: 'Med.',
    4: 'Med.',
    5: 'Low',
    6: 'Low'
}

abs_order = ['', 'High', 'Med.', 'Low']

In [308]:
# Collect all prompt_id = 0
df_list = []
models = [
    'llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 
    'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-instruct','gemma-7b', 'gemma-7b-instruct', 'gemma-2-9b', 'gemma-2-9b-instruct']


# Iterate over models and files, and read CSVs into DataFrame
for model, file_name in product(models, ['explicit.csv', 'implicit.csv']):
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df['explicit'] = ('explicit' in file_name)
    df['model'] = model

    # Filter only rows where 'debiasing_prompt_id' is 0
    df = df[df['debiasing_prompt_id'] == 0]

    if 'explicit' in file_name:
        # Calculate the averages for male, female, and diverse columns if available (explicit)
        male_cols = [f"male_explicit{i}" for i in range(25) if f"male_explicit{i}" in df.columns]
        female_cols = [f"female_explicit{i}" for i in range(25) if f"female_explicit{i}" in df.columns]
        diverse_cols = [f"diverse_explicit{i}" for i in range(25) if f"diverse_explicit{i}" in df.columns]
    else:
        # Calculate the averages for male, female, and diverse columns if available (implicit)
        male_cols = [f"male_implicit{i}" for i in range(25) if f"male_implicit{i}" in df.columns]
        female_cols = [f"female_implicit{i}" for i in range(25) if f"female_implicit{i}" in df.columns]
        diverse_cols = [f"diverse_implicit{i}" for i in range(25) if f"diverse_implicit{i}" in df.columns]


    # Compute averages if there are columns to average
    if male_cols:
        df['male_avg'] = df[male_cols].mean(axis=1)
    if female_cols:
        df['female_avg'] = df[female_cols].mean(axis=1)
    if diverse_cols:
        df['diverse_avg'] = df[diverse_cols].mean(axis=1)

    # Append the processed DataFrame to the list
    df_list.append(df)

# Concatenate all DataFrames
df_default = pd.concat(df_list)

# Group by 'model', 'explicit', and 'female_dominated', then calculate averages
grouped = df_default.groupby(['model', 'explicit', 'female_dominated']).agg(
    male_avg=('male_avg', 'mean'),
    female_avg=('female_avg', 'mean'),
    diverse_avg=('diverse_avg', 'mean')
).reset_index()

# Save the new averaged DataFrame
averages_df = grouped

In [309]:
# Load the aggregated DataFrame "averages_df"
models = [
    'llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 
    'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-instruct',  
    'gemma-7b', 'gemma-7b-instruct', 'gemma-2-9b', 'gemma-2-9b-instruct'
]

# Generate LaTeX table
latex_table = r'''
\begin{table*}[ht!]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    \caption{Results for all models.}
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & \multicolumn{6}{c}{Explicit} & \multicolumn{6}{c}{Implicit} \\
    \cmidrule(lr){2-7} \cmidrule(lr){8-13}
    Model & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
    & M & F & D & M & F & D & M & F & D & M & F & D \\
    \midrule
'''

def calculate_average(df, gender_prefix):
    """Calculate the average of all columns that start with the given gender prefix."""
    columns = [col for col in df.columns if col.startswith(gender_prefix)]
    if not df.empty and len(columns) > 0:
        return df[columns].mean(axis=1).values[0]
    return None

for model in models:
    model_name = model_str_map[model]  # Escape hyphens for LaTeX
    explicit_fd = averages_df[(averages_df['model'] == model) & (averages_df['explicit'] == True) & (averages_df['female_dominated'] == True)]
    explicit_md = averages_df[(averages_df['model'] == model) & (averages_df['explicit'] == True) & (averages_df['female_dominated'] == False)]
    implicit_fd = averages_df[(averages_df['model'] == model) & (averages_df['explicit'] == False) & (averages_df['female_dominated'] == True)]
    implicit_md = averages_df[(averages_df['model'] == model) & (averages_df['explicit'] == False) & (averages_df['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not explicit_fd.empty:
        male_avg = explicit_fd['male_avg'].values[0]
        female_avg = explicit_fd['female_avg'].values[0]
        diverse_avg = explicit_fd['diverse_avg'].values[0]
        row += f"{male_avg*100:.1f}\\% & {female_avg*100:.1f}\\% & {diverse_avg*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not explicit_md.empty:
        male_avg = explicit_md['male_avg'].values[0]
        female_avg = explicit_md['female_avg'].values[0]
        diverse_avg = explicit_md['diverse_avg'].values[0]
        row += f"{male_avg*100:.1f}\\% & {female_avg*100:.1f}\\% & {diverse_avg*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not implicit_fd.empty:
        male_avg = implicit_fd['male_avg'].values[0]
        female_avg = implicit_fd['female_avg'].values[0]
        diverse_avg = implicit_fd['diverse_avg'].values[0]
        row += f"{male_avg*100:.1f}\\% & {female_avg*100:.1f}\\% & {diverse_avg*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not implicit_md.empty:
        male_avg = implicit_md['male_avg'].values[0]
        female_avg = implicit_md['female_avg'].values[0]
        diverse_avg = implicit_md['diverse_avg'].values[0]
        row += f"{male_avg*100:.1f}\\% & {female_avg*100:.1f}\\% & {diverse_avg*100:.1f}\\% "
    else:
        row += " & & &"
    
    row += r"\\"
    if "instruct" in model and model != models[-1]:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\label{tab:explicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results_debias/default.tex


In [310]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results.tex')

for model in models:
    model_name = model.replace('-', '')
    model_description = model_str_map[model]
    # Load the CSV file into a DataFrame
    df_implicit = pd.read_csv(os.path.join(results_dir, model, 'implicit.csv'))
    df_explicit = pd.read_csv(os.path.join(results_dir, model, 'explicit.csv'))
    
    for col in df_implicit.columns:
        if 'implicit' in col:
            df_implicit.rename(columns={col:col.replace('implicit','')},inplace=True)
    for col in df_explicit.columns:
        if 'explicit' in col:
            df_explicit.rename(columns={col:col.replace('explicit','')},inplace=True)

    # Calculate means for each combination of prompt_id, conversation, and female_dominated
    grouped_df_implicit = df_implicit.groupby(['model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()    
    grouped_df_explicit = df_explicit.groupby(['model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()    

    # Drop the 'model' column
    model_df_implicit = grouped_df_implicit.drop(columns=['model'])
    model_df_explicit = grouped_df_explicit.drop(columns=['model'])
    
    # Sort by Abs. and prompt_id to ensure correct order
    model_df_implicit['debiasing_prompt_id'] = model_df_implicit['debiasing_prompt_id'].astype(int)
    model_df_implicit = model_df_implicit.sort_values(by=['debiasing_prompt_id'])
    
    model_df_explicit['debiasing_prompt_id'] = model_df_explicit['debiasing_prompt_id'].astype(int)
    model_df_explicit = model_df_explicit.sort_values(by=['debiasing_prompt_id'])
    
    model_df_implicit['explicit'] = False
    model_df_explicit['explicit'] = True
    
    model_df = pd.concat([model_df_implicit, model_df_explicit])

In [311]:
model_df

,debiasing_prompt_id,conversation,female_dominated,male_0_prob,female_0_prob,diverse_0_prob,male_0,female_0,diverse_0,male_1_prob,...,male_23,female_23,diverse_23,male_24_prob,female_24_prob,diverse_24_prob,male_24,female_24,diverse_24,explicit
0,0,False,False,0.000049,0.000018,0.000017,0.639935,0.197265,0.162795,0.000497,...,0.80822,0.175630,0.016145,0.000015,0.000003,0.000001,0.844600,0.078855,0.076540,False
1,0,False,True,0.000003,0.000075,0.000011,0.067075,0.832875,0.100040,0.000086,...,0.15274,0.832780,0.014460,0.000003,0.000068,0.000009,0.082110,0.812855,0.105055,False
0,0,False,False,0.847224,0.000017,0.001996,0.949330,0.000070,0.050600,0.658988,...,0.96328,0.001460,0.035255,0.992564,0.000002,0.000340,0.999615,0.000000,0.000385,True
1,0,False,True,0.046521,0.886191,0.002237,0.051335,0.945780,0.002890,0.007729,...,0.06166,0.876085,0.062255,0.049127,0.942991,0.000082,0.050065,0.949845,0.000080,True
2,1,False,False,0.168421,0.004225,0.010939,0.702165,0.037390,0.260450,0.021094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,1,False,True,0.047479,0.143553,0.011032,0.240985,0.517625,0.241390,0.004447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2,False,False,0.507391,0.000679,0.000648,0.951400,0.004695,0.043885,0.374531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
5,2,False,True,0.014372,0.639140,0.000566,0.087720,0.899780,0.012500,0.007123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
6,3,False,False,0.333336,0.004590,0.010771,0.792485,0.020075,0.187450,0.376171,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
7,3,False,True,0.036420,0.281209,0.014362,0.127405,0.670645,0.201955,0.017251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [312]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results.tex')

with open(output_tex_file, 'w') as f_out:
    f_out.write(r'\onecolumn')
    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')
        # Load the CSV file into a DataFrame
        df_implicit = pd.read_csv(os.path.join(results_dir, model, 'implicit.csv'))
        df_explicit = pd.read_csv(os.path.join(results_dir, model, 'explicit.csv'))
        

        # Calculate means for each combination of prompt_id, conversation, and female_dominated
        grouped_df_implicit = df_implicit.groupby(['model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()    
        grouped_df_explicit = df_explicit.groupby(['model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()    

        # Drop the 'model' column
        model_df_implicit = grouped_df_implicit.drop(columns=['model'])
        model_df_explicit = grouped_df_explicit.drop(columns=['model'])
        
        # Sort by Abs. and prompt_id to ensure correct order
        model_df_implicit['debiasing_prompt_id'] = model_df_implicit['debiasing_prompt_id'].astype(int)
        model_df_implicit = model_df_implicit.sort_values(by=['debiasing_prompt_id'])
        
        model_df_explicit['debiasing_prompt_id'] = model_df_explicit['debiasing_prompt_id'].astype(int)
        model_df_explicit = model_df_explicit.sort_values(by=['debiasing_prompt_id'])
        
        model_df_implicit['explicit'] = False
        model_df_explicit['explicit'] = True
        
        model_df = pd.concat([model_df_implicit, model_df_explicit])

        latex_table = r'''
\begin{table*}[ht!]
\centering
\small
% Reduce text size and slightly the gap between columns
% \setlength{\tabcolsep}{4.6pt} % Default: 5pt
\caption{Results for ''' + model_str_map[model] + r''' on debiasing prompts.}
  \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
\begin{tabular}{c c c c c c c c c c c c c c}
\toprule
& \multicolumn{6}{c}{Explicit} & \multicolumn{6}{c}{Implicit} \\
\cmidrule(lr){2-7} \cmidrule(lr){8-13}
& \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
     ID & M & F & D & M & F & D & M & F & D & M & F & D\\
    \midrule
'''



        for prompt_id, prompt_group in model_df.groupby('debiasing_prompt_id'):
            id_label = 'None' if prompt_id == 0 else str(prompt_id)
            row_prefix = f"        {id_label} & "

            explicit_fd = prompt_group[(prompt_group['explicit'] == True) & (prompt_group['female_dominated'] == True)]
            explicit_md = prompt_group[(prompt_group['explicit'] == True) & (prompt_group['female_dominated'] == False)]
            implicit_fd = prompt_group[(prompt_group['explicit'] == False) & (prompt_group['female_dominated'] == True)]
            implicit_md = prompt_group[(prompt_group['explicit'] == False) & (prompt_group['female_dominated'] == False)]

            row = row_prefix

            if not explicit_fd.empty:
                row += f"{explicit_fd['male'].values[0]*100:.1f}\\% & {explicit_fd['female'].values[0]*100:.1f}\\% & {explicit_fd['diverse'].values[0]*100:.1f}\\% & "
            else:
                row += " & & & "

            if not explicit_md.empty:
                row += f"{explicit_md['male'].values[0]*100:.1f}\\% & {explicit_md['female'].values[0]*100:.1f}\\% & {explicit_md['diverse'].values[0]*100:.1f}\\% & "
            else:
                row += " & & & "

            if not implicit_fd.empty:
                row += f"{implicit_fd['male'].values[0]*100:.1f}\\% & {implicit_fd['female'].values[0]*100:.1f}\\% & {implicit_fd['diverse'].values[0]*100:.1f}\\% & "
            else:
                row += " & & & "

            if not implicit_md.empty:
                row += f"{implicit_md['male'].values[0]*100:.1f}\\% & {implicit_md['female'].values[0]*100:.1f}\\% & {implicit_md['diverse'].values[0]*100:.1f}\\% "
            else:
                row += " & & "

            row += r"\\"
            latex_table += row + "\n"


        latex_table += r'''
\bottomrule
\end{tabular}
 } % end \resizebox
\label{tab:''' + model_name + r'''_debias}
\end{table*}

'''

        f_out.write(latex_table)
        # f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)

KeyError: 'male'

In [286]:
model_df

,debiasing_prompt_id,conversation,female_dominated,male_implicit0_prob,female_implicit0_prob,diverse_implicit0_prob,male_implicit0,female_implicit0,diverse_implicit0,male_implicit1_prob,...,diverse_explicit23_prob,male_explicit23,female_explicit23,diverse_explicit23,male_explicit24_prob,female_explicit24_prob,diverse_explicit24_prob,male_explicit24,female_explicit24,diverse_explicit24
0,0,False,False,0.000886,0.000101,0.000016,0.87981,0.102205,0.01797,0.00137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,False,True,0.000304,0.000306,0.000016,0.45324,0.525920,0.02085,0.00034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003017,0.727270,0.260405,0.012305,0.224377,0.053976,0.002224,0.799655,0.19227,0.008085
1,0,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003391,0.441395,0.544035,0.014585,0.087161,0.169080,0.002481,0.350665,0.63906,0.010300


In [313]:
from itertools import product
import pandas as pd
import os

# Collect all prompt_id = 0
df_0 = []



for model, file_name in product(models, ['non_gq.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])

df_0 = pd.concat(df_0)

# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c}
    \toprule
    & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    # do not add midrule if model is the last in models
    if "instruct" in model and model != models[-1]:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on implicit bias (average over task prompts 2-4).}
\label{tab:implicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'implicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)
    
    

    # Save the LaTeX table to a file
# output_tex_file = os.path.join(results_dir, 'default.tex')
# with open(output_tex_file, 'w') as f_out:
#     f_out.write(latex_table_explicit)
#     f_out.write('\n\n')
#     f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)


FileNotFoundError: [Errno 2] No such file or directory: '../data/results_debias/llama3-8b/non_gq.csv'